# Dask

In [4]:
import kaggle
import zipfile
import dask.dataframe as dd
from dask.distributed import Client

In [2]:
kaggle.api.authenticate()
kaggle.api.dataset_download_files('marklvl/bike-sharing-dataset', path='data', unzip=False)

In [3]:
zipper = zipfile.ZipFile('data/bike-sharing-dataset.zip', 'r')
zipper.extractall('data/')
zipper.close()

In [ ]:
client = Client()
client

In [15]:
df = dd.read_csv('data/hour.csv')

In [20]:
df

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
npartitions=1,,,,,,,,,,,,,,,,,
,int64,object,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [17]:
client.close()